# Part 1

The CPU itself is a large, black building surrounded by a bottomless pit. Enormous metal tubes extend outward from the side of the building at regular intervals and descend down into the void. There's no way to cross, but you need to get inside.

No way, of course, other than building a bridge out of the magnetic components strewn about nearby.

Each component has two ports, one on each end. The ports come in all different types, and only matching types can be connected. You take an inventory of the components by their port types (your puzzle input). Each port is identified by the number of pins it uses; more pins mean a stronger connection for your bridge. A `3/7` component, for example, has a type-3 port on one side, and a type-7 port on the other.

Your side of the pit is metallic; a perfect surface to connect a magnetic, zero-pin port. Because of this, the first port you use must be of type 0. It doesn't matter what type of port you end with; your goal is just to make the bridge as strong as possible.

The strength of a bridge is the sum of the port types in each component. For example, if your bridge is made of components `0/3`, `3/7`, and `7/4`, your bridge has a strength of `0+3 + 3+7 + 7+4 = 24`.

For example, suppose you had the following components:

```
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
```

With them, you could make the following valid bridges:

```
0/1
0/1--10/1
0/1--10/1--9/10
0/2
0/2--2/3
0/2--2/3--3/4
0/2--2/3--3/5
0/2--2/2
0/2--2/2--2/3
0/2--2/2--2/3--3/4
0/2--2/2--2/3--3/5
```

(Note how, as shown by `10/1`, order of ports within a component doesn't matter. However, you may only use each port on a component once.)

Of these bridges, the strongest one is `0/1--10/1--9/10`; it has a strength of `0+1 + 1+10 + 10+9 = 31`.

What is the strength of the strongest bridge you can make with the components you have available?

In [8]:
from io import StringIO
from textwrap import dedent

import toolz as tz

In [3]:
def _test_input():
    data = StringIO(dedent("""\
        0/2
        2/2
        2/3
        3/4
        3/5
        0/1
        10/1
        9/10
        """))
    
    return tuple(tuple(int(x) for x in row.split('/')) for row in data)
TEST_INPUT = _test_input()

In [6]:
def _puzzle_input():
    with open('./inputs/day24/input.txt') as f:
        return tuple(tuple(int(x) for x in row.split('/')) for row in f)
PUZZLE_INPUT = _puzzle_input()

In [48]:
def matches(to_match, components, available):
    for idx, i in enumerate(available):
        if to_match in components[i]:
            matched_ind = components[i].index(to_match)
            not_matched_ind = 0 if matched_ind == 1 else 1
            yield i, idx, components[i][not_matched_ind], components[i]

def build(to_match, components, available, bridge):
    any_matched = False
    for _, idx, not_matched, match in matches(to_match, components, available):
        any_matched = True
        next_available = available[:idx] + available[idx + 1:]
        yield from build(
            not_matched, components, next_available, bridge + (match,))
    if not any_matched:
        yield bridge

In [49]:
tz.first(build(0, TEST_INPUT, tuple(range(len(TEST_INPUT))), tuple()))

((0, 2), (2, 2), (2, 3), (3, 4))

In [50]:
bridges = build(0, TEST_INPUT, tuple(range(len(TEST_INPUT))), tuple())
max(bridges, key=lambda b: sum(tz.concat(b)))

((0, 1), (10, 1), (9, 10))

In [51]:
tuple(build(0, TEST_INPUT, tuple(range(len(TEST_INPUT))), tuple()))

(((0, 2), (2, 2), (2, 3), (3, 4)),
 ((0, 2), (2, 2), (2, 3), (3, 5)),
 ((0, 2), (2, 3), (3, 4)),
 ((0, 2), (2, 3), (3, 5)),
 ((0, 1), (10, 1), (9, 10)))

In [52]:
bridges = build(0, PUZZLE_INPUT, tuple(range(len(PUZZLE_INPUT))), tuple())
strongest = max(bridges, key=lambda b: sum(tz.concat(b)))
strongest

((0, 39),
 (39, 30),
 (30, 35),
 (41, 35),
 (32, 41),
 (3, 32),
 (17, 3),
 (46, 17),
 (46, 50),
 (41, 50),
 (49, 41),
 (49, 4),
 (4, 30),
 (44, 30),
 (40, 44),
 (23, 40),
 (38, 23),
 (38, 20),
 (20, 24),
 (24, 14),
 (14, 45),
 (47, 45),
 (47, 11),
 (45, 11),
 (45, 50),
 (50, 36),
 (36, 31),
 (5, 31),
 (5, 5),
 (44, 5),
 (29, 44),
 (29, 29),
 (29, 12))

In [53]:
sum(tz.concat(strongest))

2006

# Part 2

The bridge you've built isn't long enough; you can't jump the rest of the way.

In the example above, there are two longest bridges:

```
0/2--2/2--2/3--3/4
0/2--2/2--2/3--3/5
```

Of them, the one which uses the `3/5` component is stronger; its strength is `0+2 + 2+2 + 2+3 + 3+5 = 19`.

What is the strength of the longest bridge you can make? If you can make multiple bridges of the longest length, pick the strongest one.

In [56]:
bridges = build(0, TEST_INPUT, tuple(range(len(TEST_INPUT))), tuple())
strongest_longest = max(bridges, key=lambda b: (len(b), sum(tz.concat(b))))
strongest_longest

((0, 2), (2, 2), (2, 3), (3, 5))

In [58]:
bridges = build(0, PUZZLE_INPUT, tuple(range(len(PUZZLE_INPUT))), tuple())
strongest_longest = max(bridges, key=lambda b: (len(b), sum(tz.concat(b))))
strongest_longest

((0, 39),
 (39, 30),
 (30, 35),
 (41, 35),
 (32, 41),
 (3, 32),
 (17, 3),
 (2, 17),
 (2, 31),
 (36, 31),
 (50, 36),
 (41, 50),
 (49, 41),
 (49, 4),
 (4, 30),
 (44, 30),
 (44, 5),
 (5, 5),
 (5, 37),
 (20, 37),
 (38, 20),
 (38, 23),
 (23, 40),
 (40, 24),
 (24, 14),
 (14, 45),
 (47, 45),
 (47, 11),
 (45, 11),
 (45, 50),
 (46, 50),
 (46, 17),
 (22, 17),
 (22, 48))

In [59]:
sum(tz.concat(strongest_longest))

1994